In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import shutil

Remove `1.6/` folder from `unzippedFaces`

In [2]:
# basedir = '/media/storage8tb/data/voxceleb1/unzippedFaces/'

# list_of_dirs = os.listdir(basedir)

# for folder in tqdm(list_of_dirs):
#     root = basedir + folder
    
#     for folder2 in os.listdir(root + '/1.6/'):
#         shutil.move(root + '/1.6/' + folder2, os.path.join(root, folder2))
        
#     os.rmdir(root + '/1.6/')

Rename folders

In [3]:
meta = pd.read_csv('/media/storage8tb/data/voxceleb1/metadata/vox1_meta.csv', sep='\t')
meta.head()

,VoxCeleb1 ID,VGGFace1 ID,Gender,Nationality,Set
0,id10001,A.J._Buckley,m,Ireland,dev
1,id10002,A.R._Rahman,m,India,dev
2,id10003,Aamir_Khan,m,India,dev
3,id10004,Aaron_Tveit,m,USA,dev
4,id10005,Aaron_Yoo,m,USA,dev


Name2id

In [4]:
# name2id = meta.set_index('VGGFace1 ID')['VoxCeleb1 ID'].to_dict()

# ### AUDIO ###
# basedir = '/media/storage8tb/data/voxceleb1/audio/'

# list_of_dirs = os.listdir(basedir)

# for folder in tqdm(list_of_dirs):
#     root = basedir + folder
#     prev_name = folder
#     new_name = name2id[folder]
#     os.rename(basedir + prev_name, basedir + new_name)
    
# ### VIDEO ###
# basedir = '/media/storage8tb/data/voxceleb1/video/'

# list_of_dirs = os.listdir(basedir)

# for folder in tqdm(list_of_dirs):
#     root = basedir + folder
#     prev_name = folder
#     new_name = name2id[folder]
#     os.rename(basedir + prev_name, basedir + new_name)

id2name

In [5]:
# id2name = meta.set_index('VoxCeleb1 ID')['VGGFace1 ID'].to_dict()

# ### AUDIO ###
# basedir = '/media/storage8tb/data/voxceleb1/audio/'

# list_of_dirs = os.listdir(basedir)

# for folder in tqdm(list_of_dirs):
#     root = basedir + folder
#     prev_name = folder
#     new_name = id2name[folder]
#     os.rename(basedir + prev_name, basedir + new_name)

# ### VIDEO ###
# basedir = '/media/storage8tb/data/voxceleb1/video/'

# list_of_dirs = os.listdir(basedir)

# for folder in tqdm(list_of_dirs):
#     root = basedir + folder
#     prev_name = folder
#     new_name = id2name[folder]
#     os.rename(basedir + prev_name, basedir + new_name)

Remove tracks from `voice_set_labels.txt` (i.e. dataset) which does not have frames in `zippedFaces.tar.gz`. Note, that the _"we should never have more than 20 frames in a sequence from the same track"_ is taken into account. 

In [6]:
path_to_data = '/media/storage8tb/data/voxceleb1/'
path_to_split = '/media/storage8tb/data/voxceleb1/Splits/voice_set_labels.txt'

In [7]:
# ## outputs a file `still_missing_frames.txt` with missing frames

# still_missing_frames_savepath = '/home/vladimir/Desktop/still_missing_frames.txt'

# still_missing_frames = []

# voice_set_labels = pd.read_table(path_to_split, sep=' ', names=['path', 'phase'])
# voice_set_labels.replace({'_000': '/0', '.wav$': ''}, inplace=True, regex=True)

# for path in tqdm(voice_set_labels['path']):
#     video_path = os.path.join(path_to_data, 'video', path + '.txt')
    
#     if os.path.isfile(os.path.join(path_to_data, 'audio', path + '.wav')):
#         frames = pd.read_table(video_path, skiprows=6, usecols=['FRAME '])
        
#     earliest = frames['FRAME '].iloc[0]
#     latest = frames['FRAME '].iloc[-1]
#     frame_list = np.arange(earliest, latest+1)
#     mask = np.where(frame_list % 25 == 0)
#     frames_sec = frame_list[mask]
#     # only 20 per each face-track (see the asterics on the project page)
#     # frames_sec = frame_list[mask]
#     frames_sec = frame_list[mask][:20]
    
#     for frame_number in frames_sec:
#         filename ='{0:07d}.jpg'.format(frame_number)
#         selected_frame_path = os.path.join(path_to_data, 'video', path[:-5] + filename)
        
#         if os.path.isfile(selected_frame_path):
#             continue
            
#         else:
#             still_missing_frames.append(selected_frame_path[39:])
            
# with open(still_missing_frames_savepath, 'w') as out_f:
#     for file in still_missing_frames:
#         out_f.write(file + '\n')

In [8]:
## outputs unique tracks that have at least one frame missing. 

tracks_to_filter_path = '/home/vladimir/Desktop/tracks_to_filter.txt'

tracks_to_filter = []

voice_set_labels = pd.read_table(path_to_split, sep=' ', names=['path', 'phase'])
voice_set_labels.replace({'_000': '/0', '.wav$': ''}, inplace=True, regex=True)

for path in tqdm(voice_set_labels['path']):
    video_path = os.path.join(path_to_data, 'video', path + '.txt')
    
    if os.path.isfile(os.path.join(path_to_data, 'audio', path + '.wav')):
        frames = pd.read_table(video_path, skiprows=6, usecols=['FRAME '])
        
    earliest = frames['FRAME '].iloc[0]
    latest = frames['FRAME '].iloc[-1]
    frame_list = np.arange(earliest, latest+1)
    mask = np.where(frame_list % 25 == 0)
    frames_sec = frame_list[mask]
    # only 20 per each face-track (see the asterics on the project page)
    # frames_sec = frame_list[mask]
    frames_sec = frame_list[mask][:20]
    
    for frame_number in frames_sec:
        filename ='{0:07d}.jpg'.format(frame_number)
        selected_frame_path = os.path.join(path_to_data, 'video', path[:-5] + filename)
        
        if os.path.isfile(selected_frame_path):
            continue
            
        else:
            # remove dir path and file extention
            tracks_to_filter.append(video_path[39:])
            break
            
with open(tracks_to_filter_path, 'w') as out_f:
    for file in tracks_to_filter:
        out_f.write(file + '\n')

100%|██████████| 153486/153486 [03:45<00:00, 679.58it/s]


In [41]:
153486 - len(tracks_to_filter)

153333

In [39]:
mask = voice_set_labels['path'].apply(lambda x: x in tracks_to_filter)
indices = voice_set_labels[mask].index

filteredpath = os.path.join(path_to_data, 'Splits', 'filtered_voice_set_labels.txt')

with open(path_to_split, 'r') as r, open(filteredpath, 'w') as w:
    
    for idx, line in enumerate(r.readlines()):
        
        if idx not in indices:
            w.write(line)
        
        else:
            continue

153486it [00:00, 485618.85it/s]


In [ ]:
## which phases the missing files belong to (1 and 3 -- train and seen test)
print(voice_set_labels[mask]['phase'].value_counts())

In [13]:
'Corbin_Bleu/fXwfYa_uUZw/00003' in tracks_to_filter

False

In [15]:
## remove these tracks from the `voice_set_labels.txt` and write the rest to a new file

filteredpath = os.path.join(path_to_data, 'Splits', 'filtered_voice_set_labels.txt')
i = 0
with open(path_to_split, 'r') as r, open(filteredpath, 'w') as w:
    
    for line in r.readlines():
        path, _ = line.split('.wav ')
        print(path)
        i += 1
        
        if i > 3:
            break

A.J._Buckley/Y8hIVOBuels_0000001
A.J._Buckley/Y8hIVOBuels_0000002
A.J._Buckley/Y8hIVOBuels_0000003
A.J._Buckley/Y8hIVOBuels_0000004
